In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy

/Users/noahkasmanoff/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
#Reading in all data, tab delimited
playByPlayDf = pd.read_csv('Basketball Analytics/Play_by_Play.txt',sep='\t')
lineupsDf = pd.read_csv('Basketball Analytics/Game_Lineup.txt',sep='\t')
codesDf = pd.read_csv('Basketball Analytics/Event_Codes.txt',sep='\t')

In [5]:
def find_code(msg_type,action_type=None):
    if action_type:
        return codesDf[(codesDf['Event_Msg_Type'] == msg_type) & (codesDf['Action_Type']==action_type)]
    else:
        return codesDf[codesDf['Event_Msg_Type'] == msg_type]

In [6]:
codesDf.Event_Msg_Type.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 18,
       20])

##### Code descriptions
1: Made Shot <br>
2: Missed Shot <br>
3: Free Throw (Except Action Type == 0)<br>
6: Foul<br>
8: Substitution<br>

###### Sort order for play by play per instructions
Period (ascending), PC_Time (descending), WC_Time (ascending),
Event_Number (ascending)

In [7]:
print(lineupsDf.columns)
print('')
print(playByPlayDf.columns)

Index(['Game_id', 'Period', 'Person_id', 'Team_id', 'status'], dtype='object')

Index(['Game_id', 'Event_Num', 'Event_Msg_Type', 'Period', 'WC_Time',
       'PC_Time', 'Action_Type', 'Option1', 'Option2', 'Option3', 'Team_id',
       'Person1', 'Person2', 'Person3', 'Team_id_type', 'Person1_type',
       'Person2_type', 'Person3_type'],
      dtype='object')


In [8]:
playByPlayDf.sort_values('Period',ascending=True,inplace=True)
playByPlayDf.sort_values('PC_Time',ascending=False,inplace=True)
playByPlayDf.sort_values('WC_Time',ascending=True,inplace=True)
playByPlayDf.sort_values('Event_Num',ascending=True,inplace=True)
playByPlayDf.reset_index(inplace=True)
playByPlayDf.drop(axis=0,columns=['index'],inplace=True)

In [9]:
freeThrows = playByPlayDf[playByPlayDf.Event_Msg_Type == 3].index

In [10]:
pbpArray = np.array(playByPlayDf)
for idx in freeThrows:
    prevIdx = idx - 1
    while pbpArray[prevIdx,2] == 8:
        pbpArray[[prevIdx,idx]] = pbpArray[[idx,prevIdx]]
        prevIdx -= 1

In [11]:
subSwap = pd.DataFrame(pbpArray,columns = playByPlayDf.columns)

In [12]:
subSwap.loc[(playByPlayDf.Event_Msg_Type == 3) & (playByPlayDf.Option1 == 2),'Option1'] = 0

In [13]:
subSwap['onthefloor'] = ''
stacked = pd.DataFrame()
for gameId in lineupsDf['Game_id'].unique():
    gameLineups = lineupsDf[lineupsDf['Game_id'] == gameId]
    gamePbp = subSwap[subSwap['Game_id'] == gameId]
    gamePbp = gamePbp[gamePbp['Team_id'].isin(gameLineups['Team_id'].unique())]
    currDict = dict()
    for team in gameLineups['Team_id'].unique():
        currDict[team] = []
        currTeam = set(gameLineups[gameLineups['Team_id'] == team]['Person_id'].unique())
        lastIter=set()
        while len(currTeam) != len(lastIter):
            lastIter=deepcopy(currTeam)
            currTeam.update(
                gamePbp[
                    (gamePbp['Event_Msg_Type'] == 8) & 
                    (gamePbp['Person1'].isin(currTeam))
                ]['Person2'].unique()
            )
        teamCount = len(currTeam)
        currDf = pd.DataFrame(
            list(zip(currTeam,[0] * teamCount, [gameId] * teamCount,[team] * teamCount)),
            columns=['playerid','plus_minus','gameid','teamid']
        )
        stacked = pd.concat([stacked,currDf])
    subSwap.loc[subSwap['Game_id'] == gameId,'onthefloor'] = [currDict]

In [14]:
subSwap.columns

Index(['Game_id', 'Event_Num', 'Event_Msg_Type', 'Period', 'WC_Time',
       'PC_Time', 'Action_Type', 'Option1', 'Option2', 'Option3', 'Team_id',
       'Person1', 'Person2', 'Person3', 'Team_id_type', 'Person1_type',
       'Person2_type', 'Person3_type', 'onthefloor'],
      dtype='object')

In [15]:
stacked.columns

Index(['playerid', 'plus_minus', 'gameid', 'teamid'], dtype='object')

In [16]:
vals

NameError: name 'vals' is not defined

In [25]:
subSwapNew = subSwap.reset_index().copy()
prevPeriod = -1
prevTeamDict = dict()
count = 1
plusMinus = stacked.copy()
for gameId in ['006728e4c10e957011e1f24878e6054a']:
    print(f'Finding data for game {count} out of {len(subSwapNew.Game_id.unique())}')
    count+=1
    currGame = subSwapNew[subSwapNew.Game_id == gameId]
    for idx, vals in currGame.iterrows():
        if vals.Period != prevPeriod:
            prevPeriod = vals.Period
            for team in vals.onthefloor.keys():
                subSwapNew.iloc[vals['index']].onthefloor[team] = list(
                    lineupsDf[
                        (lineupsDf['Game_id'] == gameId) & 
                        (lineupsDf['Team_id'] == team) & 
                        (lineupsDf['Period'] == vals.Period)

                    ].Person_id
                )
        elif vals.Event_Msg_Type == 8:
            for team in vals.onthefloor.keys():
                if vals.Person1 in vals.onthefloor[team]:
                    playerIdx = subSwapNew.iloc[vals['index']].onthefloor[team].index(vals.Person1)
                    subSwapNew.iloc[vals['index']].onthefloor[team][playerIdx] = vals.Person2
        else:
            subSwapNew.loc[vals['index'],'onthefloor'] = [prevTeamDict]
        
        prevTeamDict = subSwapNew.iloc[vals['index']].onthefloor
        
        if vals.Event_Msg_Type in [1,3]:
            points = vals.Option1
            scoringTeam = vals.Team_id
            for team in vals.onthefloor.keys():
                if len(vals.onthefloor[team]) != 5:
                    print(vals.onthefloor[team])
                for player in vals.onthefloor[team]:
                    if team == scoringTeam:
                        plusMinus.loc[
                            (plusMinus.playerid == player) & (plusMinus.gameid == vals.Game_id),
                            'plus_minus'] += points
                    else:
                        plusMinus.loc[
                            (plusMinus.playerid == player) & (plusMinus.gameid == vals.Game_id),
                            'plus_minus'] -= points
            if plusMinus.plus_minus.sum() != 0:
                print('ERROR')
        
        for team in prevTeamDict.keys():
            if len(prevTeamDict[team]) != 5:
                print (vals['index'],gameId,prevTeamDict)

Finding data for game 1 out of 82


,playerid,plus_minus,gameid,teamid
1,618f6d58ab2881152607c2a6e057bc51,1,006728e4c10e957011e1f24878e6054a,01be0ad4af7aeb1f6d2cc2b6b6d6d811
2,2ad626904c8b28cceb8e12c624a84240,-12,006728e4c10e957011e1f24878e6054a,01be0ad4af7aeb1f6d2cc2b6b6d6d811
3,42e0d7167f04a4ff958c6442da0e6851,-2,006728e4c10e957011e1f24878e6054a,01be0ad4af7aeb1f6d2cc2b6b6d6d811
4,5db9c1c8184510fee8161e7fafdc9c49,-8,006728e4c10e957011e1f24878e6054a,01be0ad4af7aeb1f6d2cc2b6b6d6d811
7,f4a5ca938177c407a9dab5412e39498f,-6,006728e4c10e957011e1f24878e6054a,01be0ad4af7aeb1f6d2cc2b6b6d6d811
9,766802a8fda500d7945950de7398c9c6,-6,006728e4c10e957011e1f24878e6054a,01be0ad4af7aeb1f6d2cc2b6b6d6d811
11,ae53f8ba6761b64a174051da817785bc,-4,006728e4c10e957011e1f24878e6054a,01be0ad4af7aeb1f6d2cc2b6b6d6d811
12,8d2127290c94bd41b82a2938734bc750,-10,006728e4c10e957011e1f24878e6054a,01be0ad4af7aeb1f6d2cc2b6b6d6d811
13,c5dd5b2e3b975f0849d9b74e74125cb9,-13,006728e4c10e957011e1f24878e6054a,01be0ad4af7aeb1f6d2cc2b6b6d6d811
2,48ec4e6c52f418d5ca4ef510ba473ea0,3,006728e4c10e957011e1f24878e6054a,45ba8fc87f55b1191c50c400dc7ed11c


In [431]:
finalAnswer = plusMinus.rename(columns={'playerid':'Player_ID','plus_minus':'Player_Plus/Minus','gameid':'Game_ID'}).drop(columns='teamid')

In [433]:
finalAnswer = finalAnswer[['Game_ID','Player_ID','Player_Plus/Minus']]

In [ ]:
finalAnswer.to_csv('Q1_BBALL.csv')